In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

import warnings

# Disbale all warning messages
warnings.filterwarnings('ignore')

In [2]:
#Read data from documents (PDF)
DATA_PATH = 'D:/Data/AIO/Chatbot QnA/chatbotqna/data/YOLOv10_Tutorials.pdf'
loader = PyPDFLoader(DATA_PATH)
# loader = Loader(DATA_PATH)
documents = loader.load()

In [3]:
#Init text split

txt_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

txt_docs = txt_spliter.split_documents(documents)

print(f"Number pf chunks: {len(txt_docs)}")
print(f"Print first doc: {txt_docs[0]}")

Number pf chunks: 33
Print first doc: page_content='AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I. Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổ điển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tự động xác định vị trí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quả bài toán này. Trong đó, các phương pháp thuộc họ YOLO (You Only Look
Once) thu hút được sự chú ý rất lớn từ cộng đồng nghiên cứu bởi độ chính xác và tốc độ thực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sự tại Đại học Thanh Hoa (Tsinghua University)
đã đề xuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object
Detection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu

In [4]:
# Initial Vetorization:
embeding = HuggingFaceEmbeddings()
vector_db = Chroma.from_documents(documents=txt_docs, embedding=embeding)
retriver = vector_db.as_retriever()

In [5]:
# Testing conversation

results = retriver.invoke("What is YOLOv10?")
print(len(results))

4


In [6]:
# Build LLM
#Initial cfg for LLM
nf4_cfg = BitsAndBytesConfig(load_in_4bit=True,\
                            bnb_4bit_quant_type="nf4", \
                            bnb_4bit_use_double_quant=True,\
                            bnb_4bit_compute_dtype=torch.float16)

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

llm_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, quantization_config = nf4_cfg, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

llm_model_pipline = pipeline("text-generation",\
                            model=llm_model,\
                            tokenizer=tokenizer,\
                            max_new_tokens=512,\
                            pad_token_id = tokenizer.eos_token_id,\
                            device_map="auto")#{"": "cuda:0"})

llm_hugg_pipline = HuggingFacePipeline(pipeline=llm_model_pipline)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Run test

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriver | format_docs, "question": RunnablePassthrough()}
    |prompt
    |llm_hugg_pipline
    |StrOutputParser()
)

# USER_QUESTION = "YOLOv10 là gì?"
# output = rag_chain.invoke(USER_QUESTION)
# answer = output.split("Answer:")[1].strip()
# print(answer)

In [8]:
USER_QUESTION = "YOLOv10 là gì? Cách sử dụng như thế nào? Cho vi du."
output = rag_chain.invoke(USER_QUESTION)
answer = output.split("Answer:")[1].strip()
print(answer)

YOLOv10 là một phiên bản của mô hình dự đoán hình ảnh YOLO (You Only Look Once) với tối ưu hóa tham số và độ trễ trong inference. Nó có mức độ chính xác tốt hơn so với các phiên bản trước và sử dụng PGI và GELAN để cải thiện hiệu suất và độ chính xác. Để sử dụng pre-trained models, các bạn cần tải về file trọng số (file.pt) và khởi tạo mô hình bằng cách sử dụng đoạn code của ultralytics. Sau đó, các bạn có thể tải ảnh cần dự đoán vào Colab tự động bằng cách sử dụng đoạn code của Google.
